In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import networkit as nk
import collections
import os
seed = 0
graph_name = "karate"
output_path = "../output/QTM/"
input_path = "../input/"
graph_path = input_path + graph_name + ".graph"
result_path = output_path + graph_name + ".txt"
if not os.path.exists(output_path):
    os.makedirs(output_path)
result_file = open(result_path,"w+")

nk.setSeed(seed, False)
result_file.write("Using seed " + str(seed) + "\n")

G = nk.readGraph(graph_path, nk.Format.METIS)
nk.viztasks.drawGraph(G)
G.indexEdges()
editing = nk.community.QuasiThresholdEditingLinear(G)
editing.run()
parents = []
parentNames = []
parents.append([i for i in range(G.upperNodeIdBound())])
parentNames.append("Trivial")
parents.append(editing.getParents())
parentNames.append("Editing")

orders = []
orderNames = []
orders.append([])
orderNames.append("None")
orders.append([i for i in range(G.upperNodeIdBound())])
orderNames.append("By id")

degree_dict = {}
for u in G.iterNodes():
    degree_dict[u] = G.degree(u)
orders.append([el[0] for el in sorted(degree_dict.items(), key=lambda kv: kv[1])])
orderNames.append("Degree ascending")

runs = [0, 2, 5, 20]
plateaus = [1, 5, 10]


for i in range(len(parents)):
    for j in range(len(orders)):
        if(i > 0 and j > 0): 
            break
        for run in runs:
            sortPaths = False
            for k in range(2):
                randomness = True
                for l in range(2):
                    if(randomness):
                        for plateau in plateaus:
                            mover = nk.community.QuasiThresholdEditingLocalMover(G, parents[i], run, sortPaths, randomness, orders[j], plateau)
                            mover.run()
                            result_file.write("Parents: " + parent_names[i])
                            result_file.write(", Order: "+ order_names[j])
                            result_file.write(", Runs: "+ str(run))
                            result_file.write(", sortPaths: ", str(sortPaths))
                            result_file.write(", randomness: ", str(randomness))
                            result_file.write(", plateauSize: ", str(plateau))
                            result_file.write("----- Edits: ", str(mover.getNumberOfEdits())),
                            result_file.write(" used ", str(mover.getUsedIterations()))
                            #result_file.write(" actualPlateau ", str(mover.getPlateauSize()))
                            result_file.write("\n")
                    else:
                        mover = nk.community.QuasiThresholdEditingLocalMover(G, parents[i], run, sortPaths, randomness, orders[j], plateau)
                        mover.run()
                        result_file.write("Parents: " + parent_names[i])
                        result_file.write(", Order: "+ order_names[j])
                        result_file.write(", Runs: "+ str(run))
                        result_file.write(", sortPaths: ", str(sortPaths))
                        result_file.write(", randomness: ", str(randomness))
                        result_file.write("----- Edits: ", str(mover.getNumberOfEdits())),
                        result_file.write(" used ", str(mover.getUsedIterations()))
                        result_file.write("\n")
                            
result_file.close()

ImportError: /home/luise/Bachelorarbeit/networkit/_NetworKit.cpython-35m-x86_64-linux-gnu.so: undefined symbol: _ZNK9NetworKit31QuasiThresholdEditingLocalMover8getCoverEm